# Demonstration of pyQ3 python wrapper for DEW-ENKI-EQ3 aqueous speciation calculations - TUTORIAL 1
**Important: before running any calculations you must have a compiled set of DEW species objects. If you are running this on your own machine you should run the import_DEW.ipynb notebook before running this one.**

First import the required packages:

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import pyQ3
from thermoengine import model, phases, chem
%run core.ipynb
import dill

## Create a custom DATA0 file
One use for the code is to automatically create a customised DATA0 file for use with EQPT and EQ3 externally. This can use the full set of aqueous species, or a custom system can be specified.

### Generic DATA0 file at a chosen pressure
The DEW DATA0 files available online are for specific pressures, but pyQ3 allows DATA0 to be generated for any chosen pressure and any temperature range.

First you must define the DEW system. To obtain the default species and minerals you do not need to specify anything when defining the system:

In [26]:
dew_system = pyQ3.System()

To generate the DATA0 file, you must call the create_data0() method. The temperature (in K) you supply will be the starting temperature of the file, and it will cover 350K in excess of this temperature. The pressure is constant.

In [28]:
T = 300.0 + 273.15 # in K
P = 10000.0 # in bar
dew_system.make_data0(T,P)

The new DATA0 file will now be in the directory you are running the notebook in. Note that the nomenclature used for the species is not identical to that used in the published DATA0 files, instead it is closer to the nomenclature used in the DEW spreadsheet.

### Create a customised DATA0 file - subsystem
Perhaps you wish to add only a small selection of complexes, or you want to include some DEW complexes that aren't included by default. We can start by looking at which species, elements and minerals are included by default:

In [34]:
print("The elements included by default:")
print(pyQ3.defaultsystem.elements)
print("\nThe basis species used by default:")
print(pyQ3.defaultsystem.basis_species_names)
print("\nAll the species included by default:")
print(pyQ3.defaultsystem.other_species_names)

The elements included by default:
['O', 'H', 'Ag', 'Al', 'Au', 'Ba', 'Ca', 'C', 'Si', 'Cl', 'S', 'Cd', 'N', 'Cr', 'Cs', 'Cu', 'Eu', 'F', 'Fe', 'P', 'Hg', 'K', 'Mg', 'Mn', 'Na', 'Pb', 'Sr', 'U', 'Zn']

The basis species used by default:
['H2O', 'H+', 'AG+', 'AL+3', 'AU+', 'BA+2', 'CA+2', 'CO2(AQ)', 'SIO2(AQ)', 'CL-', 'SO4-2', 'CD+2', 'N2(AQ)', 'CR+2', 'CS+', 'CU+', 'EU+2', 'F-', 'FE+2', 'H3PO4(AQ)', 'HG+2', 'K+', 'MG+2', 'MN+2', 'NA+', 'PB+2', 'SR+2', 'U+4', 'ZN+2', 'O2']

All the species included by default:
['ACETATE(AQ)', 'ACETIC(AQ)', 'AG+', 'AL+3', 'ALO2-', 'ALO2(SIO2)-', 'AU+', 'BA+2', 'CA(HCO3)+', 'CA(HCOO)+', 'CA(HSIO3)+', 'CA(OH)+', 'CA+2', 'CACL+', 'CACO3(AQ)', 'CASO4(AQ)', 'CD+2', 'CL-', 'CN-', 'CO(AQ)', 'CO2(AQ)', 'CO3-2', 'CR+2', 'CS+', 'CU+', 'CU+2', 'DIGLYCINE(AQ)', 'DIKETOPIP(AQ)', 'ETHANE(AQ)', 'ETHANOL(AQ)', 'ETHYLENE(AQ)', 'EU+2', 'F-', 'FE(HCOO)+', 'FE(OH)+', 'FE+2', 'FE+3', 'FECL+', 'FECL+2', 'FECL2(AQ)', 'FECL3(AQ)', 'FECL4-', 'FEO(AQ)', 'FORMATE(AQ)', 'FORMIC(AQ)'

We can also see which minerals are included by default. However, note that we use mineral abbreviations, rather than mineral names, when setting up the DEW system:

In [35]:
print(pyQ3.defaultsystem.minerals)

['Ap', 'Mag', 'Crn', 'Hem', 'Per', 'Lm', 'Brc', 'Dsp', 'Cal', 'Mgs', 'And', 'Ky', 'Sil', 'Lws', 'Zo', 'cZo', 'Fo', 'Fa', 'Mtc', 'Mw', 'Ctl', 'cEn', 'En', 'pEn', 'Di', 'Jd', 'Fs', 'Wo', 'Tr', 'Ath', 'Crd', 'Or', 'Ab', 'An', 'Gh', 'Kln', 'Atg', 'Prl', 'Tlc', 'Ms', 'Phl', 'Pg', 'Prh', 'Clc', 'Prp', 'Alm', 'Grs', 'Mei', 'Qz', 'Coe']


The solid solutions use the full mineral names. Note that the DATA0 can only have mineral and species names < 14 characters long. Some mineral names are automatically truncated:

In [37]:
pyQ3.defaultsystem.solid_solutions

{'K-FELDSPAR(SS)': ['POTASSIUM_FEL', 'ALBITE'],
 'PLAGIOCLASE(SS)': ['ALBITE', 'ANORTHITE'],
 'ORTHOPYROXENE(SS)': ['ORTHOENSTATIT', 'FERROSILITE'],
 'OLIVINE(SS)': ['FORSTERITE', 'FAYALITE'],
 'GARNET(SS)': ['PYROPE', 'ALMANDINE', 'GROSSULAR'],
 'CLINOPYROXENE(SS)': ['DIOPSIDE', 'JADEITE']}

But let's say we are only interested in calculating the solubility of quartz. We must define the system accordingly:

In [44]:
elements = ['O','H','Si']
basis_species = ['H2O','H+','SIO2(AQ)','O2']
other_species = ['SI2O4(AQ)','SI3O6(AQ)','HSIO3-','O2(AQ)','OH-']
minerals = ['Qz']
solid_solutions = {}

Note that H2O, H+ and O2 must be included in the basis species. OH- must be included in other_species.

In [45]:
dew_system = pyQ3.System(elements = elements,
                         basis_species = basis_species,
                         other_species = other_species,
                         minerals = minerals,
                         solid_solutions = solid_solutions)

In [47]:
T = 300.0 + 273.15 # in K
P = 8000.0 # in bar
dew_system.make_data0(T,P)

### Create a customised DATA0 file - other species
Any species that exists within the DEW database can be added to the system as above, but how to tell what species are available? Well we can import the DEW database directly into the Jupyter notebook:

In [48]:
load_coder_modules(working_dir='dew2019_coderfiles')
with open('DEW2019.pkl','rb') as file:
    dewdb = dill.load(file)
    file.close()
print('Number of species = '+str(len(dewdb)))

Number of species = 228


In [51]:
s = ''
i = 0
for speciesname in dewdb:
    s += speciesname
    s += ', '
    if i == 9:
        s +='\n'
        i = 0
    else:
        i += 1
print(s)

ACETATE(AQ), ACETIC(AQ), AG+, AGCL(AQ), AGCL2-, AL+3, ALO2-, ALO2(SIO2)-, AR(AQ), AU+, 
AU+3, B(OH)3(AQ), BA+2, BACL+, BE+2, BENZENE(AQ), BO(OH)(AQ), BO2-, BR-, CA(HCO3)+, 
CA(HCOO)+, CA(HSIO3)+, CA(OH)+, CA+2, CACL+, CACL2(AQ), CACO3(AQ), CAO(AQ), CASO4(AQ), CD+2, 
CE+3, CE+4, CL-, CN-, CO(AQ), CO+2, CO+3, CO2(AQ), CO3-2, CR+2, 
CR+3, CR2O7-2, CRO4-2, CS+, CU+, CU+2, DIGLYCINE(AQ), DIKETOPIP(AQ), DY+3, ER+3, 
ETHANE(AQ), ETHANOL(AQ), ETHYLENE(AQ), EU+2, EU+3, F-, FE(CH3COO)+, FE(CH3COO)2(A, FE(HCOO)+, FE(HSIO3)+, 
FE(OH)+, FE+2, FE+3, FECL+, FECL+2, FECL2(AQ), FECL2+, FECL3(AQ), FECL4-, FEO(AQ), 
FORMATE(AQ), FORMIC(AQ), GA+3, GD+3, GLUTAMATE-, GLUTAMIC(AQ), GLUTAMINE(AQ), GLUTARATE-, GLUTARIC(AQ), GLYCINATE-, 
GLYCINE(AQ), GLYCOLATE-, GLYCOLIC(AQ), H-SUCCINA(AQ), H+, H2(AQ), H2ASO3-, H2ASO4-, H2CO3(AQ), H2P2O7-2, 
H2PO4-, H2S(AQ), H2VO4-, H3P207-, H3PO4(AQ), HALO2(AQ), HASO4-2, HCL(AQ), HCN(AQ), HCO3-, 
HCRO4-, HE(AQ), HEXANE(AQ), HFEO2-, HG+2, HG2+2, HO+3, HO2-, HPO4-2, HS-, 
HSE-, 

Let's say we want to calculate Zn speciation in a fluid in equilibrium with Nepheline. There are several Zn complexes that exist in DEW but are not included by default (but they should perhaps be used with care). Nepheline is also not included by default. The DEW database is calibrated relative to the Berman thermodynamic database, and so we must use a model for Nepheline that is compatible with Berman. Luckily Berman does include Nepheline! But we must import Berman directly into the Jupyter Notebook:

In [52]:
berman = model.Database()

To see all the mineral phases available:

In [53]:
print(berman.phase_info.to_string())

   abbrev          phase_name            formula phase_type endmember_num
0      Bt             Biotite                      solution             2
1     Cam      Clinoamphibole                      solution             3
2     Cpx       Clinopyroxene                      solution             7
3     Fsp            Feldspar                      solution             3
4     Grt              Garnet                      solution             3
5     Hbl          Hornblende                      solution             3
6    KlsS        Kalsilite ss                      solution             4
7    LctS             Leucite                      solution             3
8     Mll            Melilite                      solution             4
9    NphS        Feldspathoid                      solution             4
10     Ol             Olivine                      solution             6
11    Opx       Orthopyroxene                      solution             7
12   OrOx          OrthoOxide         

To retrieve Nepheline (note that the abbreviation must be used):

In [54]:
Nph = berman.get_phase('Nph')

Let's create the system! Note that you can use either strings, or the phase/species objects themselves when defining a system with Berman and DEW. Passing the objects themselves is particularly useful when using your own models of minerals and species. Here I will pass ZNCL+ as an object, to demonstrate this. I will also pass the Nph object directly:

In [59]:
elements = ['O','H','Si','Al','Na','Zn']
basis_species = ['H2O','H+','SIO2(AQ)','AL+3','NA+','ZN+2','O2']
other_species = ['OH-','ALO2-','ALO2(SIO2)-','HALO2(AQ)','NAOH(AQ)', dewdb['ZNCL+'], 'ZNCL2(AQ)', 'ZNCL3-','O2(AQ)']
minerals = ['Qz',Nph]
solid_solutions = {}

In [60]:
dew_system = pyQ3.System(elements = elements,
                         basis_species = basis_species,
                         other_species = other_species,
                         minerals = minerals,
                         solid_solutions = solid_solutions)

In [61]:
T = 300.0 + 273.15 # in K
P = 8000.0 # in bar
dew_system.make_data0(T,P)